# Created by Jacek Grzybowski

## Part 1

In [173]:
# import libraries
import cv2
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Activation, MaxPooling2D, Dropout
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
from sklearn.utils import shuffle
from pathlib import Path
import os

In [191]:
# load images as grayscale 64 by 64 
cloudyPath = Path('weather/cloudy').glob('*.jpg')
rainPath = Path('weather/rain').glob('*.jpg')
shinePath = Path('weather/shine').glob('*.jpg')
sunrisePath = Path('weather/sunrise').glob('*.jpg')

def toDataSetImg(path):
    img = cv2.imread(path)
    img.resize((64, 64, 3))
    return img

cloudyAll = []
for p in cloudyPath:
    cloudyAll.append(toDataSetImg(str(p)))

rainAll = []
for p in rainPath:
    rainAll.append(toDataSetImg(str(p)))

shineAll = []
for p in shinePath:
    shineAll.append(toDataSetImg(str(p)))

sunriseAll = []
for p in sunrisePath:
    sunriseAll.append(toDataSetImg(str(p)))

In [175]:
#check if the datasets have the same size
print("cloudy:", len(cloudyAll))
print("rain:", len(rainAll))
print("shine:", len(shineAll))
print("sunrise", len(sunriseAll))

cloudy: 300
rain: 212
shine: 252
sunrise 356


In [176]:
# divide to test and train dataset with ratio 25 / 75
maxSize = 200 #min(len(cloudyAll), len(rainAll), len(shineAll), len(sunriseAll))
trainSize = int(maxSize * 0.75)

trainCloud = cloudyAll[0: trainSize]
trainYCloud = ["cloud"] * len(trainCloud)
testCloud = cloudyAll[trainSize-1: -1]
testYCloud = ["cloud"] * len(testCloud)

trainSunrise = sunriseAll[0: trainSize]
trainYSunrise = ["sunrise"] * len(trainSunrise)
testSunrise = sunriseAll[trainSize-1: -1]
testYSunrise = ["sunrise"] * len(testSunrise)

trainRain = rainAll[0: trainSize]
trainYRain = ["rain"] * len(trainRain)
testRain = rainAll[trainSize-1: -1]
testYRain = ["rain"] * len(testRain)

trainShine = shineAll[0: trainSize]
trainYShine = ["shine"] * len(trainShine)
testShine = shineAll[trainSize-1: -1]
testYShine = [] * len(testShine)


In [177]:
# create end train/test dataset

X_train = []
X_train.extend(trainCloud)
X_train.extend(trainSunrise)
X_train = np.array(X_train)

y_train = []
y_train.extend(trainYCloud)
y_train.extend(trainYSunrise)
y_train = keras.utils.to_categorical(y_train)

X_test = []
X_test.extend(testCloud)
X_test.extend(testSunrise)
X_test = np.array(X_test)

y_test = []
y_test.extend(testYCloud)
y_test.extend(testYSunrise)
y_test = keras.utils.to_categorical(y_test)



print(X_train.shape)
print(y_train.shape)


(300, 64, 64, 3)
(300, 2)


In [178]:
# create and compie model
model = Sequential()
model.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dense(2))
model.add(Activation("softmax"))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [179]:
# train model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=12)

Epoch 1/12
10/10 [==============================] - 2s 132ms/step - loss: 2.3159 - accuracy: 0.6400 - val_loss: 3.6346 - val_accuracy: 0.5646
Epoch 2/12
10/10 [==============================] - 1s 118ms/step - loss: 1.0763 - accuracy: 0.7600 - val_loss: 9.4456 - val_accuracy: 0.5758
Epoch 3/12
10/10 [==============================] - 1s 115ms/step - loss: 0.9123 - accuracy: 0.7600 - val_loss: 6.5320 - val_accuracy: 0.6685
Epoch 4/12
10/10 [==============================] - 1s 117ms/step - loss: 0.8136 - accuracy: 0.8067 - val_loss: 5.0715 - val_accuracy: 0.5843
Epoch 5/12
10/10 [==============================] - 1s 139ms/step - loss: 0.6400 - accuracy: 0.8400 - val_loss: 1.0808 - val_accuracy: 0.7753
Epoch 6/12
10/10 [==============================] - 2s 188ms/step - loss: 0.3510 - accuracy: 0.8400 - val_loss: 2.6375 - val_accuracy: 0.6826
Epoch 7/12
10/10 [==============================] - 2s 189ms/step - loss: 0.5696 - accuracy: 0.8433 - val_loss: 0.6050 - val_accuracy: 0.7556
Epoch 

In [180]:
# results
testResults = model.predict(X_test)

print(confusion_matrix(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print(classification_report(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print("Cohen's Kappa: {}".format(cohen_kappa_score(y_test.argmax(axis=1), testResults.argmax(axis=1))))
print("Accuracy: ",accuracy_score(y_test.argmax(axis=1), testResults.argmax(axis=1)))

[[132  18]
 [ 52 154]]
              precision    recall  f1-score   support

           0       0.72      0.88      0.79       150
           1       0.90      0.75      0.81       206

    accuracy                           0.80       356
   macro avg       0.81      0.81      0.80       356
weighted avg       0.82      0.80      0.80       356

Cohen's Kappa: 0.6088157729498933
Accuracy:  0.8033707865168539


```
[[132  18]
 [ 52 154]]
              precision    recall  f1-score   support

           0       0.72      0.88      0.79       150
           1       0.90      0.75      0.81       206

    accuracy                           0.80       356
   macro avg       0.81      0.81      0.80       356
weighted avg       0.82      0.80      0.80       356

Cohen's Kappa: 0.6088157729498933
Accuracy:  0.8033707865168539
```

## Part 2

In [181]:
# extend train and test data

X_train = []
X_train.extend(trainCloud)
X_train.extend(trainSunrise)
X_train.extend(trainRain)
X_train.extend(trainShine)
X_train = np.array(X_train)

y_train = []
y_train.extend(trainYCloud)
y_train.extend(trainYSunrise)
y_train.extend(trainYRain)
y_train.extend(trainYShine)
y_train = keras.utils.to_categorical(y_train)

X_test = []
X_test.extend(testCloud)
X_test.extend(testSunrise)
X_test.extend(testRain)
X_test.extend(testShine)
X_test = np.array(X_test)

y_test = []
y_test.extend(testYCloud)
y_test.extend(testYSunrise)
y_test.extend(testYRain)
y_test.extend(testYShine)
y_test = keras.utils.to_categorical(y_test)

y_train, X_train = shuffle(y_train, X_train)

print(X_train.shape)
print(y_train.shape)

(600, 64, 64, 3)
(600, 4)


In [182]:
# create model
model2 = Sequential()
model2.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model2.add(BatchNormalization())
model2.add(Activation("relu"))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(32, (3, 3), padding="same"))
model2.add(BatchNormalization())
model2.add(Activation("relu"))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(512))
model2.add(Activation("relu"))
model2.add(Dense(4))
model2.add(Activation("softmax"))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [183]:
# train model
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=24)

Epoch 1/24
19/19 [==============================] - 4s 167ms/step - loss: 4.7254 - accuracy: 0.4533 - val_loss: 6.8611 - val_accuracy: 0.3038
Epoch 2/24
19/19 [==============================] - 3s 171ms/step - loss: 1.0562 - accuracy: 0.5683 - val_loss: 3.1281 - val_accuracy: 0.3962
Epoch 3/24
19/19 [==============================] - 2s 132ms/step - loss: 0.8461 - accuracy: 0.6583 - val_loss: 1.8994 - val_accuracy: 0.5077
Epoch 4/24
19/19 [==============================] - 3s 133ms/step - loss: 0.7670 - accuracy: 0.6883 - val_loss: 1.6227 - val_accuracy: 0.4712
Epoch 5/24
19/19 [==============================] - 3s 160ms/step - loss: 0.6963 - accuracy: 0.7100 - val_loss: 0.9843 - val_accuracy: 0.6327
Epoch 6/24
19/19 [==============================] - 3s 134ms/step - loss: 0.5446 - accuracy: 0.8050 - val_loss: 0.9389 - val_accuracy: 0.6577
Epoch 7/24
19/19 [==============================] - 3s 140ms/step - loss: 0.5294 - accuracy: 0.8050 - val_loss: 0.9834 - val_accuracy: 0.6558
Epoch 

In [184]:
testResults = model2.predict(X_test)

print(confusion_matrix(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print(classification_report(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print("Cohen's Kappa: {}".format(cohen_kappa_score(y_test.argmax(axis=1), testResults.argmax(axis=1))))
print("Accuracy: ",accuracy_score(y_test.argmax(axis=1), testResults.argmax(axis=1)))

[[102  20  16  12]
 [ 17 153  26  10]
 [  7   3  52   0]
 [ 13  13   2  74]]
              precision    recall  f1-score   support

           0       0.73      0.68      0.71       150
           1       0.81      0.74      0.77       206
           2       0.54      0.84      0.66        62
           3       0.77      0.73      0.75       102

    accuracy                           0.73       520
   macro avg       0.71      0.75      0.72       520
weighted avg       0.75      0.73      0.74       520

Cohen's Kappa: 0.6290898641159325
Accuracy:  0.7326923076923076


```
[[102  20  16  12]
 [ 17 153  26  10]
 [  7   3  52   0]
 [ 13  13   2  74]]
              precision    recall  f1-score   support

           0       0.73      0.68      0.71       150
           1       0.81      0.74      0.77       206
           2       0.54      0.84      0.66        62
           3       0.77      0.73      0.75       102

    accuracy                           0.73       520
   macro avg       0.71      0.75      0.72       520
weighted avg       0.75      0.73      0.74       520

Cohen's Kappa: 0.6290898641159325
Accuracy:  0.7326923076923076
```